In [251]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from datetime import datetime
from copy import deepcopy
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import geopandas as gpd
import fsspec
import re

from time import sleep

# Mapbox
from mapbox import Geocoder

# Import dataset
aurora_cara = pd.read_csv("Aurora v2.1 data file - caracterización.csv")
aurora_feedback = pd.read_csv("Aurora v2.1 data file - ayudaHumanitaria.csv")
aurora_monitoreo = pd.read_csv("Aurora v2.1 data file - monitoreo.csv")

Creating the panel data from Aurora (first observation of each ID is first connection and the followings, monitorings)

In [252]:
#Merge tables of first connection 
aurora = pd.merge(aurora_cara, aurora_feedback)
aurora.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1137 entries, 0 to 1136
Data columns (total 70 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   ¿Quién entra al sistema?                                1137 non-null   object 
 1   ¿Cómo interactúa con el sistema?                        1137 non-null   object 
 2   Tipo de enganche                                        1137 non-null   object 
 3   UserId                                                  1137 non-null   int64  
 4   InteraID                                                1137 non-null   object 
 5   Inicio interacción                                      1137 non-null   object 
 6   Final interacción                                       0 non-null      float64
 7   Tiempo de interacción                                   333 non-null    float64
 8   Alguien te está ayudando a conectarte 

In [253]:
aurora['enganche']=aurora['¿Cómo interactúa con el sistema?'] 
aurora['lat']=aurora['Latitud']
aurora['lon']=aurora['Longitud']

In [254]:
#Adding the monitorings 
aurora_comple= aurora.append(aurora_monitoreo)

C:\Users\maola\AppData\Local\Temp\ipykernel_6472\3183888684.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aurora_comple= aurora.append(aurora_monitoreo)


In [255]:
# Drop observations of Aurora team phones, test registers and geographical atypical rows 


user_ids_to_remove = [311571598, 311398466, 311396734, 311361421, 311361350, 311361257, 311337494, 311325070,
                      311325038, 311272934, 310820267, 310543580, 310357249, 310191611, 308421831, 306028996,
                      310191611, 308421831, 306028996, 311725039, 311719001, 311718121, 311699383, 311696700,
                      312179120, 311965863, 311965863, 316773170, 311440316, 313260546, 316563135, 316734459,
                      317064115]

for user_id in user_ids_to_remove:
    aurora_comple = aurora_comple.drop(aurora_comple[aurora_comple.UserId == user_id].index)

In [113]:
#Variable of date in date format (for generating panel data)
#aurora_comple["Inicio interacción"] = pd.to_datetime(
#aurora_comple["Inicio interacción"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')


In [256]:
# Filling all observations of the same ID with the variable consent and actual country (variables of first connection) 
aurora_comple['País actual'] = aurora_comple.groupby('UserId')['País actual'].transform('first')
aurora_comple['Consentimiento'] = aurora_comple.groupby('UserId')['Consentimiento'].transform('first')
aurora_comple['Edad'] = aurora_comple.groupby('UserId')['Edad'].transform('first')
aurora_comple['Género'] = aurora_comple.groupby('UserId')['Género'].transform('first')
aurora_comple['País destino'] = aurora_comple.groupby('UserId')['País destino'].transform('first')
aurora_comple['Zona país'] = aurora_comple.groupby('UserId')['Zona país'].transform('first')
aurora_comple['Hay niños, niñas o adolescentes'] = aurora_comple.groupby('UserId')['Hay niños, niñas o adolescentes'].transform('first')
aurora_comple['enganche'] = aurora_comple.groupby('UserId')['enganche'].transform('first')

In [257]:
#filering by consent, cleaning geographical variables that are equal to "None" and drop the ones that can't identify
#the zone of first connection
aurora_comple=aurora_comple[aurora_comple['Consentimiento'] != 'NO'] 
aurora_comple=aurora_comple[aurora_comple['Latitud'] != "None"] 
aurora_comple=aurora_comple[aurora_comple['enganche'] != 'QR-Enganche'] 
#aurora_comple.dropna(subset=['País actual'], inplace=True)


In [258]:
# Dropping duplicates of monitorings (same answers differents days)
columns_to_consider = [col for col in aurora_comple.columns if col not in ['Inicio interacción', 'InteraID']]

# Use drop_duplicates with the specified columns to consider
aurora_comple = aurora_comple.drop_duplicates(subset=columns_to_consider)

In [259]:

aurora_comple= aurora_comple.sort_values(by=['UserId', 'Inicio interacción'])

# Reset the index for proper grouping
aurora_comple= aurora_comple.reset_index(drop=True)

# Create a new variable "Interaction_Sequence" based on the grouping
aurora_comple['Interaction_Sequence'] = aurora_comple.groupby('UserId').cumcount() + 1

default_value = 999999

In [261]:
#Rename variables 
newColumns = {  
                "¿En qué país naciste?" : 'e08_pais_',
                'Otro país de nacimiento'	: 'e09_otro_p',
                '¿En qué país iniciaste tu viaje actual?'	: 'e10_pais_',
                'Otro país de inicio'	: 'e11_otro_p',
                '¿En qué país vivías hace un año?'	 : 'e12_pais_',
                'Otro país'	: 'e13_otro_p',
}

aurora_comple = aurora_comple.rename(columns=newColumns)

In [262]:
# functions
def loadLocalJsonDoc(filepath, dataProp=''):
    """
    return deserialised json in dictionary

    Parameters
    ----------
    filepath: file location or buffer.
    dataProp: (optional) specified property to access required data
    """
    output = {}
    with open(file=filepath, mode='r', encoding='utf-8') as f:
        json_load = json.load(f)
        if (dataProp):
            output = json_load[dataProp]
        else:
            output = json_load
    return output


def changeCountriesByExpression(country, valueDict: dict[str, str]):
    output = ""
    for key, value in valueDict.items():
        match = re.match(r"^"+key+r".$", country)
        if (match):
            return value

    return output if len(output) else country


def processCountries(countries: list[str], valueDict: dict[str, str]):
    output = []
    for country in countries:
        try:
            new_country = changeCountriesByExpression(
                country=country, valueDict=valueDict)
            output.append(new_country)
        except Exception as e:
            output.append(default_value)
    return output


def getCountriesWithCoordinates(countries: list[str], geo_countries: gpd.GeoDataFrame):
    output = {}
    for country in countries:
        try:
            filtered_country = geo_countries[geo_countries["NAME"].str.lower(
            ) == country].reindex()
            centroidValue = (filtered_country.centroid).iloc[0]
            output[country] = {"x": centroidValue.x, "y": centroidValue.y}
        except Exception as e:
            print(e)
            output[country] = {"x": default_value, "y": default_value}

    return output


def getCoordinate(value: str, side: str, valueDict: dict[str, tuple[int, int]], expressionDict: dict[str, str]):
    try:
        country = changeCountriesByExpression(value, expressionDict)
        return valueDict[country][side]
    except Exception as e:
        return default_value


def processFieldCoordinates(df: pd.DataFrame, columnDict: dict[str, dict[str, str]], valueDict: dict[str, tuple[int, int]], expressionDict: dict[str, str]):
    local_df = deepcopy(df)
    for column in columnDict.keys():
        local_df[columnDict[column]["x"]] = local_df[column].str.lower().apply(
            lambda x: getCoordinate(x, "x", valueDict, expressionDict))
        local_df[columnDict[column]["y"]] = local_df[column].str.lower().apply(
            lambda x: getCoordinate(x, "y", valueDict, expressionDict))

    return local_df

def processGeocodeData(data):
    features = data['features']
    for feature in features:
        id: str = feature['id']
        match = id.startswith("country")
        if (match):
            return (feature['properties']['short_code'], feature["place_name"])

    return "zz"


def getMapboxGeocoder(token:str):
    if(token):
        return Geocoder(access_token=token)
    else: 
        raise Exception("Invalid Token")

def reverseGeocode(longitude: int, latitude: int, token: str):
    mb_geocoder = getMapboxGeocoder(token)
    response = mb_geocoder.reverse(lat=latitude, lon=longitude)
    if (response.status_code == 200):
        data = response.json()
        return data
    else:
        return None


def processReverseGeoding(data: list[tuple[int, int]], token:str):
    output = []
    for lon, lat in data:
        try:
            result = reverseGeocode(lon, lat, token)
            _decoded = processGeocodeData(result)
            output.append(_decoded)
            sleep(1)
        except:
            output.append(("zz", ""))

    return output

def addReverseGeocodedToDataFrame(df: DataFrame, token:str  ):
    local_df = deepcopy(df)
    coordinates = list(zip(list(local_df['longitude'].astype(float).to_list()), list(
    local_df['latitude'].astype(float).to_list())))
    reversed_geocoded_df = processReverseGeoding(coordinates,token)
    local_df["country_code"] = [x[0] for x in reversed_geocoded_df]
    local_df["country_name"] = [x[1] for x in reversed_geocoded_df]
    return local_df

In [263]:
available_countries = [x.lower() for x in list(set(list(aurora_comple["e08_pais_"].unique(
)) + list(aurora_comple["e10_pais_"].unique()) + list(aurora_comple["e12_pais_"].unique()))) if type(x) == str]

countries_dict = loadLocalJsonDoc("defaults/countries_dict.json")

available_countries = processCountries(available_countries, countries_dict)

# adding coordinates value
country_data_path = "simplecache::https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"

country_df = ""

with fsspec.open(country_data_path) as file:
    country_df = gpd.read_file(file)

countriesWithCoordinates = getCountriesWithCoordinates(
    available_countries, country_df)
country_column_dict = loadLocalJsonDoc("defaults/country_column_dict.json")
aurora_comple = processFieldCoordinates(
    aurora_comple, country_column_dict, countriesWithCoordinates, countries_dict)
aurora_comple['lon_eng'] = aurora_comple['lon']
aurora_comple['lat_eng'] = aurora_comple['lat']
aurora_comple['longitude'] = aurora_comple['lon']
aurora_comple['latitude'] = aurora_comple['lat']

C:\Users\maola\AppData\Local\Temp\ipykernel_6472\1938986629.py:49: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroidValue = (filtered_country.centroid).iloc[0]


single positional indexer is out-of-bounds


In [264]:
MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")
# This is heavy process that takes a while to finish
# should be used sparingly and closer to end processes.
aurora_comple = addReverseGeocodedToDataFrame(aurora_comple, MAPBOX_TOKEN)

In [273]:
excel_file = 'completa.xlsx'
aurora_comple.to_excel(excel_file, index=False)

Building the data base of "interacciones en la ruta" - Carto page

In [290]:
# Creating the dataset for interactions across the route

df = pd.DataFrame(aurora_comple, columns = ['UserId', 'Latitud', 'Longitud','Inicio interacción', 'Interaction_Sequence'])



In [291]:
from datetime import datetime, timezone

def toUnixTimestamp(time):
    possible_formats = ["%Y-%m-%d %H:%M:%S.%f+00:00", "%d/%m/%Y %H:%M:%S.%f+00:00"]

    for format_str in possible_formats:
        try:
            # Parse the date with the specified format
            target = datetime.strptime(time, format_str)

            # Convert the datetime to UTC if it doesn't have a timezone
            if target.tzinfo is None:
                target = target.replace(tzinfo=timezone.utc)

            # Calculate the Unix timestamp in milliseconds
            in_seconds = (target - datetime(1970, 1, 1, tzinfo=timezone.utc)).total_seconds()
            in_milliseconds = int(in_seconds) * 1000

            return in_milliseconds
        except ValueError:
            continue

df["timeunix"] = df["Inicio interacción"].apply(toUnixTimestamp)

In [292]:
#renaming variables
newColumns = {
    'UserId' : 'id',
    'Latitud' : 'lat', 
    'Longitud' : 'lon',
    'Inicio interacción' : 'date',

}

df = df.rename(columns=newColumns)

In [293]:
df["date"] = pd.to_datetime(
df["date"], format='%Y-%m-%d %H:%M:%S', errors='coerce', utc=True).dt.strftime('%Y-%m-%d')


In [294]:
#sortig by time
df = df.sort_values(['id', 'timeunix'], ascending=[True, True])
df['idx'] = df.groupby('id').cumcount()

In [320]:
max_sequence_df = df.groupby('id')['Interaction_Sequence'].max().reset_index()
# Merge the result back to the original DataFrame based on 'id'
df = pd.merge(df, max_sequence_df, on='id', how='left', suffixes=('', '_max'))

# Rename the new column to 'max_seq'
df = df.rename(columns={'Interaction_Sequence_max': 'max_seq'})

In [304]:
# adding coordinates value
country_data_path = "simplecache::https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip"

country_df = ""

with fsspec.open(country_data_path) as file:
    country_df = gpd.read_file(file)


df['lon_eng'] = df['lon']
df['lat_eng'] = df['lat']
df['longitude'] = df['lon']
df['latitude'] = df['lat']

MAPBOX_TOKEN = os.environ.get("MAPBOX_TOKEN")
# This is heavy process that takes a while to finish
# should be used sparingly and closer to end processes.
df = addReverseGeocodedToDataFrame(df, MAPBOX_TOKEN)

In [322]:
#creating the structure of the variables pivoted


df['lat_idx'] = 'lat_' + df['idx'].astype(str)
df['lon_idx'] = 'lon_' + df['idx'].astype(str)

lat = df.pivot(index='id', columns='lat_idx', values='lat')
lon = df.pivot(index='id', columns='lon_idx', values='lon')

# Merge the pivoted DataFrames
reshape = pd.concat([lat, lon], axis=1).reset_index()

# Add the other columns
additional_columns = ['timeunix', 'country_name_eng', 'max_seq', 'country_code']
reshape = pd.merge(reshape, df.groupby('id')[additional_columns].first().reset_index(), on='id')

print(reshape)

             id         lat_0        lat_1 lat_10 lat_11       lat_2 lat_3  \
0     310942422   -20.4027556   -20.402715    NaN    NaN  -20.402808   NaN   
1     310947317  -20.40290667          NaN    NaN    NaN         NaN   NaN   
2     310951256           NaN          NaN    NaN    NaN         NaN   NaN   
3     310953613   -20.4027542  -20.4027542    NaN    NaN         NaN   NaN   
4     310986561    -20.402927          NaN    NaN    NaN         NaN   NaN   
...         ...           ...          ...    ...    ...         ...   ...   
1022  321587583   -18.3104707          NaN    NaN    NaN         NaN   NaN   
1023  321588497  -18.31037331          NaN    NaN    NaN         NaN   NaN   
1024  321588675   -18.3103433          NaN    NaN    NaN         NaN   NaN   
1025  321590179   -18.3104783          NaN    NaN    NaN         NaN   NaN   
1026  321616757   -17.3784135          NaN    NaN    NaN         NaN   NaN   

     lat_4 lat_5 lat_6  ... lon_4 lon_5 lon_6 lon_7 lon_8 lon_9

In [324]:
#Create additional variables that were calculate in first round
reshape['pais_fin'] = 999999
reshape['dias'] = 999999



In [329]:
#renaming variables
newColumns = {
    'lat_0' : 'lat',
    'lat_1' : 'lat_mon2',
    'lat_2' : 'lat_mon3',
    'lat_3' : 'lat_mon4',
    'lat_4' : 'lat_mon5',
    'lat_5' : 'lat_mon6',
    'lat_6' : 'lat_mon7',
    'lat_7' : 'lat_mon8',
    'lat_8' : 'lat_mon9',
    'lat_9' : 'lat_mon10',
    'lat_10': 'lat_mon11',
    'lat_11': 'lat_mon12',
    'lon_0' : 'lon', 
    'lon_1' : 'lon_mon2', 
    'lon_2' : 'lon_mon3', 
    'lon_3' : 'lon_mon4', 
    'lon_4' : 'lon_mon5', 
    'lon_5' : 'lon_mon6', 
    'lon_6' : 'lon_mon7', 
    'lon_7' : 'lon_mon8', 
    'lon_8' : 'lon_mon9', 
    'lon_9' : 'lon_mon10', 
    'lon_10' : 'lon_mon11', 
    'lon_11' : 'lon_mon12', 
    'country_name_eng' : 'country_name',
    'max_seq' : 'max_interaction'

}

reshape = reshape.rename(columns=newColumns)

In [330]:
reshape.fillna(value=999999, inplace=True)


In [331]:
#database for Carto
reshape.to_csv('Carto_map.csv', index=False)

Dataset of general feedback 

In [235]:
feedback= aurora_comple[[ 'UserId', 'Latitud', 'Longitud', 'Cual ayuda humanitaria', 'Qué tan fácil fue acceder a la ayuda', 'Interaction_Sequence', 'País actual', 'Edad', 'Género', 
                     'Qué tan satisfecho te sientes respecto a la ayuda', 'Recomendarías la ayuda ','¿Recibiste ayuda humanitaria en el lugar actual?',
                     'País destino', 'Lugar interacción', 'Hay niños, niñas o adolescentes', 'Inicio interacción']].reset_index()

In [236]:
# Rename variables according to Carto names (first round)

newColumns = { 
                'UserId' : 'Id', 
                'Latitud' : 'Lat', 
                'Longitud' : 'Lon',
                'Cual ayuda humanitaria' : 'Ayuda', 
                'Qué tan fácil fue acceder a la ayuda' : 'Acceso', 
                'Qué tan satisfecho te sientes respecto a la ayuda' : 'Satisfaccion',
                'Recomendarías la ayuda ' : 'Recomendacion',
                '¿Recibiste ayuda humanitaria en el lugar actual?' : 'Ayuda_lugaractual',
                'Hay niños, niñas o adolescentes' : 'NNA'
}


feedback = feedback.rename(columns=newColumns)

In [237]:
feedback['Ayuda'] = feedback['Ayuda'].str.split('|')
feedback['Acceso'] = feedback['Acceso'].str.split('|')
feedback['Satisfaccion'] = feedback['Satisfaccion'].str.split('|')
feedback['Recomendacion'] = feedback['Recomendacion'].str.split('|')

df = pd.DataFrame(feedback)

df = df[df.apply(lambda row: all(row[['Ayuda', 'Acceso', 'Satisfaccion', 'Recomendacion']].str.len() == row[['Ayuda', 'Acceso', 'Satisfaccion','Recomendacion']].str.len().iloc[0]), axis=1)]

df1=df.explode(['Ayuda', 'Acceso','Satisfaccion','Recomendacion'])

df1

,index,Id,Lat,Lon,Ayuda,Acceso,Interaction_Sequence,País actual,Edad,Género,Satisfaccion,Recomendacion,Ayuda_lugaractual,País destino,Lugar interacción,NNA,Inicio interacción
3,3,310947317,-20.40290667,-70.15638333,2,1,1,Chile,23.0,Mujer,2,SI,NaN,Chile,Iquique - Chile,NO,2023-10-16
4,4,310953613,-20.4027542,-70.1562681,3,2,1,Chile,42.0,Hombre,1,SI,NO,Chile,Iquique - Chile,SI,2023-10-16
5,5,310953613,-20.4027542,-70.1562681,3,2,2,Chile,42.0,Hombre,1,SI,SI,Chile,Iquique - Chile,SI,2023-10-19
7,7,311008826,-20.4029273,-70.1562247,3,1,1,Chile,28.0,Hombre,1,SI,NaN,Chile,Iquique - Chile,NO,2023-10-16
7,7,311008826,-20.4029273,-70.1562247,2,1,1,Chile,28.0,Hombre,1,SI,NaN,Chile,Iquique - Chile,NO,2023-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,1513,319462500,4.5709,-74.2973,3,2,1,Colombia,22.0,Hombre,1,SI,SI,Chile,Ipiales - Colombia,SI,2023-11-17
1514,1514,319464136,0.8222318,-77.6355349,3,2,1,Colombia,27.0,Mujer,1,SI,SI,Venezuela,Ipiales - Colombia,SI,2023-11-17
1515,1515,319464136,2.8919344,-76.5499203,3,2,2,Colombia,27.0,Mujer,1,SI,SI,Venezuela,Santander de Quilichao - Colombia,SI,2023-11-19
1520,1520,319472849,0.8236436,-77.6358278,3,1,1,Colombia,57.0,Hombre,1,SI,SI,Perú,Ipiales - Colombia,SI,2023-11-17


In [238]:
# use dictionary for aid

nec_dict = {
1: "Agua",
2: "Alimentación o kit de alimentación", 
3: "Alojamiento temporal", 
4: "Asistencia legal",
5: "Ayuda psicológica",
6: "Dinero en efectivo", 
7: "Duchas o baños", 
8: "Educación o espacios educativos",
9: "Espacios seguros para adultos",
10: "Kit de aseo o elementos de higiene",
11: "Salud, primeros auxilios o atención médica",
12: "Transporte humanitario", 
13: "Otra" 
}

def necesidadMap(value):
    try:

        if (type(value) == int):
            return nec_dict[value]
        if (type(value) == str):
            return nec_dict[int(value)]
    except Exception as e:
        return value


df1['Ayuda'] = df1['Ayuda'].apply(lambda x: necesidadMap(x))

In [240]:
# use dictionary for access, satisfaction and recomendation 

acc_dict = {
1: "Fácil",
2: "Regular", 
3: "Difícil", 
}

sat_dict = {
1: "Mucho",
2: "Poco", 
3: "Nada", 
}


def accMap(value):
    try:

        if (type(value) == int):
            return acc_dict[value]
        if (type(value) == str):
            return acc_dict[int(value)]
    except Exception as e:
        return value

def satMap(value):
    try:

        if (type(value) == int):
            return sat_dict[value]
        if (type(value) == str):
            return sat_dict[int(value)]
    except Exception as e:
        return value

df1['Acceso'] = df1['Acceso'].apply(lambda x: accMap(x))
df1['Satisfaccion'] = df1['Satisfaccion'].apply(lambda x: satMap(x))

In [241]:
excel_file = 'ayuda.xlsx'
df1.to_excel(excel_file, index=False)

Dataset children help and feedback

In [243]:
feedback_NNA = aurora_comple[[ 'UserId', 'Latitud', 'Longitud', 'Cual ayuda humanitaria NNA', 'NNA: Qué tan fácil fue acceder a la ayuda', 'Interaction_Sequence', 'País actual', 'Edad', 'Género', 
                     'NNA: Qué tan satisfecho te sientes respecto a la ayuda', 'NNA: Recomendarías la ayuda ','¿Recibiste ayuda humanitaria en el lugar actual?',
                     'País destino', 'Lugar interacción', 'Hay niños, niñas o adolescentes', 'Inicio interacción']].reset_index()
	


In [244]:
# Rename variables 

newColumns = { 
                'UserId' : 'Id', 
                'Latitud' : 'Lat', 
                'Longitud' : 'Lon',
                'Cual ayuda humanitaria NNA' : 'Ayuda_NNA', 
                'NNA: Qué tan fácil fue acceder a la ayuda' : 'Acceso_NNA', 
                'NNA: Qué tan satisfecho te sientes respecto a la ayuda' : 'Satisfaccion_NNA',
                'NNA: Recomendarías la ayuda ' : 'Recomendacion_NNA',
                '¿Recibiste ayuda humanitaria en el lugar actual?' : 'Ayuda_lugaractual',
                'Hay niños, niñas o adolescentes' : 'NNA'
}


feedback_NNA = feedback_NNA.rename(columns=newColumns)

In [245]:
feedback_NNA['Ayuda_NNA'] = feedback_NNA['Ayuda_NNA'].str.split('|')
feedback_NNA['Acceso_NNA'] = feedback_NNA['Acceso_NNA'].str.split('|')
feedback_NNA['Satisfaccion_NNA'] = feedback_NNA['Satisfaccion_NNA'].str.split('|')
feedback_NNA['Recomendacion_NNA'] = feedback_NNA['Recomendacion_NNA'].str.split('|')

df = pd.DataFrame(feedback_NNA)

df = df[df.apply(lambda row: all(row[['Ayuda_NNA', 'Acceso_NNA', 'Satisfaccion_NNA', 'Recomendacion_NNA']].str.len() == row[['Ayuda_NNA', 'Acceso_NNA', 'Satisfaccion_NNA','Recomendacion_NNA']].str.len().iloc[0]), axis=1)]

df1=df.explode(['Ayuda_NNA', 'Acceso_NNA','Satisfaccion_NNA','Recomendacion_NNA'])

df1

,index,Id,Lat,Lon,Ayuda_NNA,Acceso_NNA,Interaction_Sequence,País actual,Edad,Género,Satisfaccion_NNA,Recomendacion_NNA,Ayuda_lugaractual,País destino,Lugar interacción,NNA,Inicio interacción
13,13,311044066,-19.2740732,-68.6188029,2,1,1,Bolivia,21.0,Hombre,1,SI,SI,Chile,Colchane - Bolivia,SI,2023-10-16
13,13,311044066,-19.2740732,-68.6188029,1,1,1,Bolivia,21.0,Hombre,1,SI,SI,Chile,Colchane - Bolivia,SI,2023-10-16
14,14,311044066,-19.2740732,-68.6188029,2,1,2,Bolivia,21.0,Hombre,1,SI,SI,Chile,Pisiga - Bolivia,SI,2023-10-24
14,14,311044066,-19.2740732,-68.6188029,1,1,2,Bolivia,21.0,Hombre,1,SI,SI,Chile,Pisiga - Bolivia,SI,2023-10-24
28,28,311106131,-18.3127384,-70.3137804,1,2,1,Chile,36.0,Mujer,2,SI,SI,Chile,Arica - Chile,SI,2023-10-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,1500,319411184,7.6916836,-72.5993218,2,1,1,Colombia,19.0,Mujer,1,SI,SI,Colombia,Bochalema - Colombia,SI,2023-11-17
1513,1513,319462500,4.5709,-74.2973,3,2,1,Colombia,22.0,Hombre,1,SI,SI,Chile,Ipiales - Colombia,SI,2023-11-17
1514,1514,319464136,0.8222318,-77.6355349,3,2,1,Colombia,27.0,Mujer,1,SI,SI,Venezuela,Ipiales - Colombia,SI,2023-11-17
1515,1515,319464136,2.8919344,-76.5499203,3,2,2,Colombia,27.0,Mujer,1,SI,SI,Venezuela,Santander de Quilichao - Colombia,SI,2023-11-19


In [246]:
# use dictionary for children aid
aid_NNA_dict = {
1: "Agua",
2: "Alimentación o kit de alimentación", 
3: "Alojamiento temporal", 
4: "Ayuda psicológica",
5: "Duchas o baños",
6: "Educación o Espacios educativos y de cuidado para niños y niñas", 
7: "Kit de aseo o elementos de higiene", 
8: "Salud, primeros auxilios o atención médica",
9: "Otra" 
}

def necesidadMap(value):
    try:

        if (type(value) == int):
            return aid_NNA_dict[value]
        if (type(value) == str):
            return aid_NNA_dict[int(value)]
    except Exception as e:
        return value


df1['Ayuda_NNA'] = df1['Ayuda_NNA'].apply(lambda x: necesidadMap(x))

In [248]:
# use dictionary for access, satisfaction and recomendation 

acc_dict = {
1: "Fácil",
2: "Regular", 
3: "Difícil", 
}

sat_dict = {
1: "Mucho",
2: "Poco", 
3: "Nada", 
}


def accMap(value):
    try:

        if (type(value) == int):
            return acc_dict[value]
        if (type(value) == str):
            return acc_dict[int(value)]
    except Exception as e:
        return value

def satMap(value):
    try:

        if (type(value) == int):
            return sat_dict[value]
        if (type(value) == str):
            return sat_dict[int(value)]
    except Exception as e:
        return value

df1['Acceso_NNA'] = df1['Acceso_NNA'].apply(lambda x: accMap(x))
df1['Satisfaccion_NNA'] = df1['Satisfaccion_NNA'].apply(lambda x: satMap(x))

In [249]:
excel_file = 'ayuda_NNA.xlsx'
df1.to_excel(excel_file, index=False)

In [ ]:
# Rename variables 

newColumns = {
    ' Principales necesidades de información' : 'neces_info',

}

df = df.rename(columns=newColumns)

In [70]:
# Necesidades de información separar 
aurora_comple['neces_info'] = aurora_comple['neces_info'].str.split('|')
prueba = aurora_comple.explode('neces_info')

In [71]:
# Use dictionary to give other values to the variable
info_dict = {
1 :  'Acceso a alberge o alojamiento temporal',
2 : 'Acceso a alimentación o kit de alimentos',
3 : 'Acceso a ropa u otros artículos de cuidado personal',
4 : 'Acceso a atención en salud y medicamentos',
5 : 'Acceso a asistencia legal',
6 : 'Ubicación de la ayuda humanitaria en la ruta',
7 : 'Seguridad en la ruta y lugares de destino',
8 : 'Información sobre politica migratoria y procesos de regularización',
9 : 'Otra'
}

def Map(value):
    try:

        if (type(value) == int):
            return info_dict[value]
        if (type(value) == str):
            return info_dict[int(value)]
    except Exception as e:
        return value


prueba['neces_info'] = prueba['neces_info'].apply(lambda x: Map(x))

In [73]:
prueba = prueba[['UserId',
               'Edad',
               'Género',
               'Lugar interacción',
               'neces_info',
               'A través de qué medios a recibido información',

               ]].reset_index()

In [74]:
excel_file = 'info.xlsx'
prueba.to_excel(excel_file, index=False)

In [ ]:
aurora['Cuáles han sido tus 3 principales necesidades'] = aurora['Cuáles han sido tus 3 principales necesidades'].str.split('|')
prueba = aurora.explode('Cuáles han sido tus 3 principales necesidades')
prueba.head()

In [ ]:
prueba["Cuáles han sido tus 3 principales necesidades"].head()

In [ ]:
# use dictionary for necessities

nec_dict = {
1: "Agua",
2: "Alimentación o kit de alimentación", 
3: "Alojamiento temporal", 
4: "Asistencia legal",
5: "Ayuda psicológica",
6: "Dinero en efectivo", 
7: "Duchas o baños", 
8: "Educación o espacios educativos",
9: "Espacios seguros para adultos",
10: "Kit de aseo o elementos de higiene",
11: "Salud, primeros auxilios o atención médica",
12: "Transporte humanitario", 
13: "Otra" 
}

def necesidadMap(value):
    try:

        if (type(value) == int):
            return nec_dict[value]
        if (type(value) == str):
            return nec_dict[int(value)]
    except Exception as e:
        return value


prueba['necesidades'] = prueba['Cuáles han sido tus 3 principales necesidades'].apply(lambda x: necesidadMap(x))

In [ ]:
excel_file = 'necesidades.xlsx'
prueba.to_excel(excel_file, index=False)

In [ ]:
#columns = aurora_carto.columns.tolist()

In [10]:
#for column in columns:
 # sns.countplot(x=column, data=aurora_carto, order = aurora_carto[column].value_counts().index)
  #plt.xticks(rotation=70, fontsize=10)
  #plt.xlabel(column, fontsize=12)
#  plt.title(f'[{column}]', fontsize=12)
  #plt.show()
  #plt.clf()

NameError: name 'columns' is not defined